<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/pytorch/basic-%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 이진 분류

binary.csv 데이터셋에서

x, y, z 의 pass 관계는 x,y,z 모두 40 이상이며 평균이 60 이상일 때 True 반환

In [1]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class CustomDataset(Dataset):
  def __init__(self, file_path):
    df = pd.read_csv(file_path)
    self.x1 = df.iloc[:, 0].values
    self.x2 = df.iloc[:, 1].values
    self.x3 = df.iloc[:, 2].values
    self.y = df.iloc[:, 3].values
    self.length = len(df)

  def __getitem__(self, index):
    x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])
    y = torch.FloatTensor([self.y[index]])
    return x, y

  def __len__(self):
    return self.length

In [3]:
class CustomModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Sequential(
        nn.Linear(3, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    x = self.layer(x)
    return x

데이터셋 불러오기

In [4]:
dataset = CustomDataset('binary.csv')
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size*0.1)
test_size = dataset_size - train_size - validation_size

데이터셋 분리 및 데이터로더

In [5]:
train_dataset, validation_dataset, test_dataset = random_split(
    dataset, [train_size, validation_size, test_size], torch.manual_seed(4))
# 데이터로더
train_dataloader = DataLoader(
    train_dataset, batch_size=64, shuffle=True, drop_last=True
)
validation_dataloader = DataLoader(
    validation_dataset, batch_size=4, shuffle=True, drop_last=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=4, shuffle=True, drop_last=True
)

장치에 올리기

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device) # 이진 교차 엔트로피(binary crossentropy)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

훈련

In [7]:
for epoch in range(10000):
  cost = 0.0

  for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad() # 기울기 초기화
    loss.backward() # 역전파 수행(기울기 계산)
    optimizer.step() # 옵티마이저에 가중치 반영

    cost += loss

  cost = cost / len(train_dataloader)

  if (epoch + 1) % 1000 == 0:
    print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

<ipython-input-2-072d5e227e06>:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])


Epoch : 1000, Model : [Parameter containing:
tensor([[ 0.0028, -0.0006,  0.0036]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([0.0949], device='cuda:0', requires_grad=True)], Cost : 0.680
Epoch : 2000, Model : [Parameter containing:
tensor([[0.0034, 0.0005, 0.0041]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.0346], device='cuda:0', requires_grad=True)], Cost : 0.666
Epoch : 3000, Model : [Parameter containing:
tensor([[0.0045, 0.0017, 0.0051]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.1609], device='cuda:0', requires_grad=True)], Cost : 0.652
Epoch : 4000, Model : [Parameter containing:
tensor([[0.0052, 0.0017, 0.0053]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.2843], device='cuda:0', requires_grad=True)], Cost : 0.636
Epoch : 5000, Model : [Parameter containing:
tensor([[0.0059, 0.0036, 0.0064]], device='cuda:0', requires_grad=True), Parameter containing:
tensor([-0.4046], 

In [8]:
with torch.no_grad():
  model.eval() # 평가모드
  for x, y in validation_dataloader:
    x = x.to(device)
    y = y.to(device)

    outputs = model(x)

    print(outputs)
    print(outputs >= torch.FloatTensor([0.5]).to(device)) # 0.5 이상은 True
    print('--------------------')

tensor([[0.6707],
        [0.7298],
        [0.7368],
        [0.7800]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.5723],
        [0.6746],
        [0.4676],
        [0.3640]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [False]], device='cuda:0')
--------------------
tensor([[0.5711],
        [0.6363],
        [0.3985],
        [0.6238]], device='cuda:0')
tensor([[ True],
        [ True],
        [False],
        [ True]], device='cuda:0')
--------------------
tensor([[0.6304],
        [0.7174],
        [0.5751],
        [0.6137]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.7460],
        [0.6951],
        [0.7555],
        [0.6498]], device='cuda:0')
tensor([[True],
        [True],
        [True],
        [True]], device='cuda:0')
--------------------
tensor([[0.8068],
        [

<ipython-input-2-072d5e227e06>:12: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  y = torch.FloatTensor([self.y[index]])
